In [ ]:
# GVATdb.csv download from https://renlab.sdsc.edu/GVATdb/
# Need to do mapping from TF name -> TF uniprot mapping, and liftover variants coordinates from hg19 -> hg38 before loading

In [2]:
import requests

In [3]:
import json

In [4]:
TF_mapping_all = dict()

# Query with TF name in proteins collection

In [58]:
# Download the proteins collection from catalog in local json file
# -> would be quicker than query from api endpoint, 
# but probably doesn't matter much here since there are only ~500 TFs to query

In [5]:
with open('igvf_proteins.json') as json_file:
    data = json.load(json_file)

In [6]:
def query_TF_id(TF_symbol, data_json):

    TF_ids = []
    responses = []
    for record in data_json:
        if record.get('name') is not None:
            if TF_symbol == record['name'].replace('_HUMAN',''):
                responses.append(record)
                for response in responses:
                    TF_ids.append(response['_key'])

    return TF_ids

In [7]:
with open('GVATdb_all_TF_list.txt') as TF_list:
    for line in TF_list:
        TF_name = line.strip()
        TF_ids = query_TF_id(TF_name,data)
        TF_mapping_all[TF_name] = TF_ids
        

## check no mapping cases

In [8]:
TF_no_mapping = []

In [9]:
for key, value in TF_mapping_all.items():
    if len(value) <1:
        TF_no_mapping.append(key)

In [10]:
TF_no_mapping

['ARNTL2',
 'ATF6',
 'BARHL1',
 'BARHL2',
 'BHLHA15',
 'BHLHB2',
 'BHLHB4',
 'BHLHB5',
 'BHLHB8',
 'BHLHE22',
 'BHLHE23',
 'BSX',
 'CART1',
 'CREB3L1',
 'CREB3L4',
 'DMRTA1',
 'DMRTC2',
 'EBF2',
 'EN1',
 'EN2',
 'ENSG00000206218',
 'ESRRA',
 'ESRRB',
 'ESRRG',
 'FERD3L',
 'FOXD4L5',
 'FOXO3A',
 'HMBOX1',
 'HOXA1',
 'HOXA10',
 'HOXA11',
 'HOXA13',
 'HOXA2',
 'HOXA4',
 'HOXA5',
 'HOXA6',
 'HOXA7',
 'HOXA9',
 'HOXB13',
 'HOXB4',
 'HOXB5',
 'HOXB7',
 'HOXB9',
 'HOXC10',
 'HOXC11',
 'HOXC4',
 'HOXC8',
 'HOXC9',
 'HOXD1',
 'HOXD10',
 'HOXD12',
 'HOXD13',
 'HOXD3',
 'HOXD4',
 'HOXD9',
 'HSFY2',
 'JUND2',
 'KIAA2018',
 'LIN28B',
 'LRRN6D',
 'MLLT7',
 'MSC',
 'MYBL1',
 'MYBL2',
 'NEUROD1',
 'NEUROD2',
 'NEUROG1',
 'NEUROG2',
 'NFATC1',
 'NFATC2',
 'NFATC3',
 'NFATC4',
 'NHLH1',
 'NHLH2',
 'NKX2-3',
 'NKX2-4',
 'NKX2-8',
 'NKX2_8',
 'NKX6-1',
 'NKX6-2',
 'NKX6_2',
 'NR2F1',
 'NR3C1',
 'NR3C2',
 'NR5A1',
 'ONECUT1',
 'ONECUT2',
 'ONECUT3',
 'PHOX2A',
 'PHOX2B',
 'PKNOX1',
 'PKNOX2',
 'POU1F1',
 '

In [11]:
len(TF_no_mapping)

168

# For those TFs with no exact name matching with protein names, query by genes/proteins catalog endpoint

In [12]:
def query_TF_id_from_gene(gene_symbol):

    data_service_url = 'https://api-dev.catalog.igvf.org/api'
    endpoint = 'genes/proteins'
    query_string = 'gene_name=' + gene_symbol
    url = data_service_url + '/' + endpoint + '?' + query_string

    responses = requests.get(url).json()
    TF_ids = []
    for response in responses:
        TF_ids.append(response['_id'])

    return TF_ids

In [13]:
TF_remapping = dict()

In [14]:
for TF in TF_no_mapping:
    TF_ids = query_TF_id_from_gene(TF)
    TF_remapping[TF] = TF_ids

In [15]:
TF_remapping

{'ARNTL2': [],
 'ATF6': ['P18850'],
 'BARHL1': ['Q9BZE3'],
 'BARHL2': ['Q9NY43'],
 'BHLHA15': ['Q7RTS1'],
 'BHLHB2': [],
 'BHLHB4': [],
 'BHLHB5': [],
 'BHLHB8': [],
 'BHLHE22': ['Q8NFJ8'],
 'BHLHE23': ['Q8NDY6'],
 'BSX': ['Q3C1V8'],
 'CART1': [],
 'CREB3L1': ['Q96BA8'],
 'CREB3L4': ['Q8TEY5'],
 'DMRTA1': ['Q5VZB9'],
 'DMRTC2': ['Q8IXT2'],
 'EBF2': ['Q9HAK2'],
 'EN1': ['Q05925'],
 'EN2': ['P19622'],
 'ENSG00000206218': [],
 'ESRRA': ['P11474'],
 'ESRRB': ['O95718'],
 'ESRRG': ['P62508'],
 'FERD3L': ['Q96RJ6'],
 'FOXD4L5': ['Q5VV16'],
 'FOXO3A': [],
 'HMBOX1': ['Q6NT76'],
 'HOXA1': [],
 'HOXA10': ['P31260'],
 'HOXA11': ['P31270'],
 'HOXA13': ['P31271'],
 'HOXA2': ['O43364'],
 'HOXA4': ['Q00056'],
 'HOXA5': ['P20719'],
 'HOXA6': ['P31267'],
 'HOXA7': ['P31268'],
 'HOXA9': ['P31269'],
 'HOXB13': ['Q92826'],
 'HOXB4': ['P17483'],
 'HOXB5': ['P09067'],
 'HOXB7': ['P09629'],
 'HOXB9': ['P17482'],
 'HOXC10': ['Q9NYD6'],
 'HOXC11': ['O43248'],
 'HOXC4': ['P09017'],
 'HOXC8': ['P31273'],
 'HOXC

In [16]:
TF_no_mapping_2 = []

In [17]:
for key, value in TF_remapping.items():
    if len(value) < 1:
        TF_no_mapping_2.append(key)
    else:
        TF_mapping_all[key] = value

In [18]:
len(TF_no_mapping_2)

27

# Still have 27 no mappings cases above -> query by alias under genes/proteins

In [19]:
def query_TF_id_from_gene_alias(gene_symbol):

    data_service_url = 'https://api-dev.catalog.igvf.org/api'
    endpoint = 'genes/proteins/'
    query_string = 'alias=' + gene_symbol
    url = data_service_url + '/' + endpoint + '?' + query_string

    responses = requests.get(url).json()
    TF_ids = []
    for response in responses:
        TF_ids.append(response['_id'])

    return TF_ids

In [20]:
TF_remapping_2 = dict()

In [21]:
for TF in TF_no_mapping_2:
    TF_ids = query_TF_id_from_gene_alias(TF)
    TF_remapping_2[TF] = TF_ids

In [22]:
TF_no_mapping_3 = []

In [23]:
for key, value in TF_remapping_2.items():
    if len(value) <1:
        TF_no_mapping_3.append(key)
    else:
        TF_mapping_all[key] = value

In [24]:
TF_no_mapping_3

['ENSG00000206218',
 'HOXA1',
 'JUND2',
 'NKX2_8',
 'NKX6_2',
 'POU3F4',
 'POU6F1',
 'ZBTB34']

# Just remaining 8 TFs missing mapping -> assign manually

In [ ]:
# skip ENSG00000206218

In [26]:
TF_mapping_all['HOXA1'] = ['P49639']
TF_mapping_all['JUND2'] = ['P17535']
TF_mapping_all['NKX2_8'] = ['O15522']
TF_mapping_all['NKX6_2'] = ['Q9C056']
TF_mapping_all['POU3F4'] = ['P49335']
TF_mapping_all['POU6F1'] = ['Q14863']
TF_mapping_all['ZBTB34'] = ['Q8NCN2']

In [27]:
for k,v in TF_mapping_all.items():
    if len(v) > 1:
        print (k,v)

CART1 ['O75154', 'Q9BUZ4', 'Q15699']


In [28]:
# manually solve multiple mapping cases

In [29]:
TF_mapping_all['CART1'] = 'Q9BUZ4'

In [30]:
TF_mapping_all

{'ALX1': ['Q15699'],
 'ALX3': ['O95076'],
 'ARGFX': ['A6NJG6'],
 'ARNT2': ['Q9HBZ2'],
 'ARNTL2': ['Q8WYA1'],
 'ARX': ['Q96QS3'],
 'ASCL1': ['P50553'],
 'ASCL2': ['Q99929'],
 'ATF2': ['P15336'],
 'ATF3': ['P18847'],
 'ATF4': ['P18848'],
 'ATF6': ['P18850'],
 'ATF6B': ['Q99941'],
 'ATOH1': ['Q92858'],
 'ATOH7': ['Q8N100'],
 'BACH2': ['Q9BYV9'],
 'BARHL1': ['Q9BZE3'],
 'BARHL2': ['Q9NY43'],
 'BARX2': ['Q9UMQ3'],
 'BATF': ['Q16520'],
 'BATF3': ['Q9NR55'],
 'BBX': ['Q8WY36'],
 'BCL6': ['P41182'],
 'BCL6B': ['Q8N143'],
 'BHLHA15': ['Q7RTS1'],
 'BHLHB2': ['O14503'],
 'BHLHB4': ['Q8NDY6'],
 'BHLHB5': ['Q8NFJ8'],
 'BHLHB8': ['Q7RTS1'],
 'BHLHE22': ['Q8NFJ8'],
 'BHLHE23': ['Q8NDY6'],
 'BSX': ['Q3C1V8'],
 'CART1': 'Q9BUZ4',
 'CDX1': ['P47902'],
 'CDX4': ['O14627'],
 'CEBPB': ['P17676'],
 'CEBPE': ['Q15744'],
 'CEBPG': ['P53567'],
 'CLOCK': ['O15516'],
 'CPSF4': ['O95639'],
 'CREB1': ['P16220'],
 'CREB3': ['O43889'],
 'CREB3L1': ['Q96BA8'],
 'CREB3L4': ['Q8TEY5'],
 'CREB5': ['Q02930'],
 'CREM': ['

## dump the TF mappings to a pkl file

In [31]:
import pickle

In [32]:
output = open('GVATdb_TF_mapping.pkl','wb')

In [33]:
pickle.dump(TF_mapping_all,output)

In [34]:
output.close()

# Merge original data table with the variant coordinates in hg38 after liftover

In [ ]:
# liftover tool downloaded from: http://hgdownload.cse.ucsc.edu/admin/exe/
# useful article about using liftover: https://genviz.org/module-01-intro/0001/06/02/liftoverTools/
# running command:
# liftover GVATdb_hg19.bed hg19ToHg38.over.chain.gz GVATdb_hg38.bed unmapped

In [36]:
import pandas as pd

In [37]:
df = pd.read_csv('GVATdb.csv')

In [38]:
df.head()

,TF,experiment,oligo,oligo_auc,oligo_pval,ref,alt,ref_auc,alt_auc,pbs,pval,fdr,rsid
0,ALX1,FL.6.0.G12,chr10:114386719-114386759,3.02599,0.00133,C,T,1.44024,-1.28154,2.72179,0.31704,1.0,rs76124550
1,ALX1,FL.6.0.G12,chr10:114386749-114386789,3.67250,0.00049,T,C,-1.62935,0.59975,-2.22910,0.40390,1.0,rs115699571
2,ALX1,FL.6.0.G12,chr10:114393556-114393596,1.06835,0.04213,T,G,2.04137,-2.32915,4.37052,0.12897,1.0,rs703332
3,ALX1,FL.6.0.G12,chr10:114619126-114619166,6.30886,0.00002,G,A,1.08776,-0.37869,1.46646,0.57072,1.0,rs74156858
4,ALX1,FL.6.0.G12,chr10:114693095-114693135,1.44146,0.02158,A,T,0.79777,-0.61519,1.41297,0.58432,1.0,rs116045311


In [39]:
df_2 = pd.read_csv('GVATdb_uniq_hg38.bed', delimiter='\t', header = None)

In [40]:
df_2.head()

,0,1,2,3,4,5
0,chr1,49979657,49979658,rs4582846,C,T
1,chr1,49991164,49991165,rs11576945,C,T
2,chr1,50025842,50025843,rs78845435,G,A
3,chr1,50048880,50048881,rs4489606,T,C
4,chr1,50050054,50050055,rs4636518,A,G


In [41]:
df_2.columns = ['chr','start','end','rsid','ref','alt']

In [42]:
df_2.head()

,chr,start,end,rsid,ref,alt
0,chr1,49979657,49979658,rs4582846,C,T
1,chr1,49991164,49991165,rs11576945,C,T
2,chr1,50025842,50025843,rs78845435,G,A
3,chr1,50048880,50048881,rs4489606,T,C
4,chr1,50050054,50050055,rs4636518,A,G


In [43]:
df_merged = df_2.merge(df)

In [44]:
df_merged.shape

(2319536, 16)

In [46]:
df_merged_uniq = df_merged.drop_duplicates()

In [48]:
df_merged_uniq.shape

(1611653, 16)

In [49]:
df_merged_uniq.head()

,chr,start,end,rsid,ref,alt,TF,experiment,oligo,oligo_auc,oligo_pval,ref_auc,alt_auc,pbs,pval,fdr
0,chr1,49979657,49979658,rs4582846,C,T,DMRTC2,FL.7.1.A7,chr1:50445310-50445350,2.80064,0.03285,0.33993,0.58504,-0.24511,0.88535,0.98584
1,chr1,49979657,49979658,rs4582846,C,T,FOXA2,DBD.8.0.B10,chr1:50445310-50445350,1.55439,0.03224,-0.46211,-0.08603,-0.37608,0.89092,0.99856
2,chr1,49979657,49979658,rs4582846,C,T,HOXA13,FL.5.1.F3,chr1:50445310-50445350,1.80979,0.02799,-2.38964,-0.00280,-2.38684,0.43601,0.99686
3,chr1,49979657,49979658,rs4582846,C,T,MAF,FL.5.1.C3,chr1:50445310-50445350,3.54352,0.00102,0.50441,-0.04574,0.55015,0.85728,0.99998
4,chr1,49979657,49979658,rs4582846,C,T,MAFG,DBD.1.1.H6,chr1:50445310-50445350,2.16224,0.00679,0.23015,0.55587,-0.32572,0.82207,0.97275


In [50]:
df_merged_uniq.to_csv('GVATdb_hg38.csv', index=False)

## novel batch csv file has a different format:

In [66]:
df = pd.read_csv('GVATdb.novel_batch.csv')

In [67]:
df.head()

,TF,snp,oligo_auc,oligo_pval,pbs,pval
0,ALX1,chr4_39045445_G_C,2.326288,1.000000,-2.505769,0.206843
1,ALX1,chr6_55207482_C_T,5.183158,0.516162,-1.226794,0.502566
2,ALX1,chr12_64238141_G_A,3.457080,0.990648,-14.455958,0.000004
3,ALX1,chr13_24789809_G_A,6.097787,0.112729,1.295581,0.342345
4,ALX1,chr4_140856533_A_G,4.003036,0.936092,0.685027,0.702173


In [68]:
df_2 = pd.read_csv('GVATdb.novel_batch.hg38_uniq.bed', delimiter='\t', header = None)

In [69]:
df_2.head()

,0,1,2,3,4,5
0,chr1,940255,940256,chr1_875636_C_T,C,T
1,chr1,940386,940387,chr1_875767_T_G,T,G
2,chr1,940389,940390,chr1_875770_A_G,A,G
3,chr1,940686,940687,chr1_876067_C_T,C,T
4,chr1,941568,941569,chr1_876949_G_C,G,C


In [70]:
df_2.columns = ['chr','start','end','snp','ref','alt']

In [71]:
df_2.head()

,chr,start,end,snp,ref,alt
0,chr1,940255,940256,chr1_875636_C_T,C,T
1,chr1,940386,940387,chr1_875767_T_G,T,G
2,chr1,940389,940390,chr1_875770_A_G,A,G
3,chr1,940686,940687,chr1_876067_C_T,C,T
4,chr1,941568,941569,chr1_876949_G_C,G,C


In [72]:
df_merged = df_2.merge(df)

In [73]:
df_merged.shape

(1055754, 11)

In [81]:
df_merged_uniq = df_merged.drop_duplicates()

In [82]:
df_merged_uniq.shape

(1047948, 11)

In [83]:
df_merged_uniq.head()

,chr,start,end,snp,ref,alt,TF,oligo_auc,oligo_pval,pbs,pval
0,chr1,940255,940256,chr1_875636_C_T,C,T,ASCL1,4.214390,3.439986e-04,1.042135,0.632781
1,chr1,940255,940256,chr1_875636_C_T,C,T,ASCL2,4.695903,1.998710e-02,0.757761,0.744490
2,chr1,940255,940256,chr1_875636_C_T,C,T,BATF3,4.389340,7.392653e-11,1.084362,0.234332
3,chr1,940255,940256,chr1_875636_C_T,C,T,CTCF,4.193749,6.496774e-02,0.610962,0.668365
4,chr1,940255,940256,chr1_875636_C_T,C,T,DBX2,2.182605,8.877244e-01,-3.620441,0.052360


In [84]:
df_merged_uniq.insert(7,'experiment','novel_batch')

In [85]:
df_merged_uniq.head()

,chr,start,end,snp,ref,alt,TF,experiment,oligo_auc,oligo_pval,pbs,pval
0,chr1,940255,940256,chr1_875636_C_T,C,T,ASCL1,novel_batch,4.214390,3.439986e-04,1.042135,0.632781
1,chr1,940255,940256,chr1_875636_C_T,C,T,ASCL2,novel_batch,4.695903,1.998710e-02,0.757761,0.744490
2,chr1,940255,940256,chr1_875636_C_T,C,T,BATF3,novel_batch,4.389340,7.392653e-11,1.084362,0.234332
3,chr1,940255,940256,chr1_875636_C_T,C,T,CTCF,novel_batch,4.193749,6.496774e-02,0.610962,0.668365
4,chr1,940255,940256,chr1_875636_C_T,C,T,DBX2,novel_batch,2.182605,8.877244e-01,-3.620441,0.052360


In [86]:
df_merged_uniq.to_csv('GVATdb_hg38.novel_batch.csv', index=False)